In [117]:
data <-read.delim('/data/pubmed-data.tsv', header=TRUE, sep="\t")

In [118]:
data_sh <- read.delim('/data/pubmed-data.tsv', header=TRUE, sep="\t", nrows=200)#sample data used to test code

In [179]:
#check sort_algorithm categories
#table(data_sh$sort_algorithm)
#added this filter to narrow down data and # of unique journals - there were over 500 unique journal names without additional filters 
data_sh=data%>% filter(grepl('asthma', query_term)&grepl('corticosteroid', query_term)) 

In [181]:
#dir.create(Sys.getenv("R_LIBS_USER"), recursive = TRUE)  # create personal library to be able to install packages
#.libPaths(Sys.getenv("R_LIBS_USER"))
#install.packages('dplyr')
library(dplyr)

In [182]:
#Change data_sh to data for final analysis. data_sh is used to test the code

In [183]:
#Filter the data to keep the search results on 1st page and sorted by data and relevance 
data_cat=dplyr::filter(data_sh, sort_algorithm=="date" |sort_algorithm=='relevance'&page_num==1)

In [184]:
#sample data from the 'population' of interest
sample=data_cat %>% group_by(sort_algorithm) %>% sample_frac(.2)# X% of rows within each sort group
#sample%>% group_by(sort_algorithm) %>% sample_n(2)# X number of rows within each sort group

In [ ]:
##create a list of lists for loop below the search by group
sort_group=sample%>%group_by(sort_algorithm, .add=TRUE)%>%group_split(sort_algorithm)%>%setNames(unique(sample$sort_algorithm))

In [186]:
#extract PMID's for the filtered data
pmIDs<-list()
for (i in names(sort_group)){
    pmIDs[[i]] <- paste(sort_group[[i]]$PMID, collapse = ",")
}

In [ ]:
#install.packages('RISmed') # package used to retrieve info for PMIDs
#may give error message - API access isn't stable
library(RISmed)
pubmed_data<-list()
records<-list()
for (i in names(pmIDs)){
records[[i]]<-EUtilsGet(pmIDs[[i]],type="efetch",db="pubmed") #retrieve the info for each PMID
#store the values of interest (3 selected for now)
pubmed_data[[i]]<- data.frame('Language' = Language(records[[i]]), 'ISOAbbreviation'= ISOAbbreviation(records[[i]]), 'YearPubDate'=YearPubDate(records[[i]]))
}
#install.packages('data.table')
library(data.table)
results=rbindlist(pubmed_data, idcol=c(names(pubmed_data)))
sum_data=results%>%group_by(date,ISOAbbreviation, YearPubDate) %>%summarize(Freq=n(),.groups = 'drop') #%>%filter(Freq >3)
#install.packages('ggplot2')
#library(ggplot2)
ggplot(data = sum_data, aes(x = ISOAbbreviation, y = Freq, fill = date)) + 
    geom_bar(stat='identity')

In [239]:
#use rentrez package to retrieve info by pmid. This is alternative to RISmed package
library(rentrez)
pubmed_data<-list()
records<-list()
for (i in names(pmIDs)){
records[[i]]<-entrez_summary(pmIDs[[i]],type="efetch",db="pubmed") #retrieve the info for each PMID
#store the values of interest (3 selected for now)
pubmed_data[[i]]<- t(extract_from_esummary(records[[i]], c("fulljournalname", "pubdate")))
}
results=do.call(rbind, Map(cbind, sort_algorithm = seq_along(pubmed_data), pubmed_data))
sum_data=data.frame(results)%>%mutate(year = substr(pubdate,1,4))%>%group_by(sort_algorithm,fulljournalname) %>%summarize(Freq=n(),.groups = 'drop') %>% dplyr::filter(!(fulljournalname==""))#%>%filter(Freq >3)

